# Trigger files
This notebook is intended to create the trigger files for the eeg data analysis.

In [1]:
import copy
import datetime
import itertools
import os

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from matplotlib.ticker import FormatStrFormatter

import pandas as pd
import pyxdf
import seaborn as sns
from scipy.signal import find_peaks
from tqdm.notebook import tqdm

In [ ]:
files = os.listdir("data")  # get all files from the folder "data"
files.sort()  # sort them alphabetically
recordings = {}
excluded = []
# for file in files:
#     # filter hidden/config files and folders
#     if not file.endswith(".xdf"):
#         files.remove(file)  # remove hidden/config file
#         excluded.append(file)  # add into excluded
#
#
# # short tests were excluded as they were too short and had strange statistics
# for e in excluded:
#     print(f"Excluded from '{e}/': {os.listdir(f'lsl_coroutines_data/{e}')}")

for i, file in enumerate(files):  # store and display all files
    created = os.path.getmtime(f"data/{file}")  # creation timestamp
    created = datetime.datetime.fromtimestamp(created)  # translate as datetime
    created = created.strftime("%d.%m.%Y %H:%M")  # arrange it
    recordings[i] = {"file": file, "created": created}

files = [f.split(".")[0] for f in files]
print("Included:")
display(recordings)